In [ ]:
!wget https://www2.census.gov/topics/genealogy/2010surnames/Names_2010Census_Top1000.xlsx

--2023-11-24 17:37:54--  https://www2.census.gov/topics/genealogy/2010surnames/Names_2010Census_Top1000.xlsx
Resolving www2.census.gov (www2.census.gov)... 104.91.110.51, 2600:1400:11:199::208c, 2600:1400:11:1b8::208c, ...
Connecting to www2.census.gov (www2.census.gov)|104.91.110.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘Names_2010Census_Top1000.xlsx’

Names_2010Census_To     [  <=>               ]  91.91K   233KB/s    in 0.4s    

2023-11-24 17:37:55 (233 KB/s) - ‘Names_2010Census_Top1000.xlsx’ saved [94119]



In [ ]:
!pip install transformers
!pip install datasets
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from datasets import load_dataset
import transformers
from transformers import pipeline, set_seed
import tokenizers
import pandas as pd
from transformers import AutoModelForCausalLM, GPT2Tokenizer

In [ ]:
dataset = load_dataset("pubmed_qa", "pqa_artificial")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
len(tokenizer.get_vocab())

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

50257

 **TODO**: Write a function generate_canaries(...) with takes in as input a list of unique names, list of unique diseases and the num of canaries to be added per record.

 - List of names are generated from the **US Census dataset 2010** which contains surnames which are ranked based on their frequency of occurrence.
  - Create full names by combining two surnames
  - Number of full names = total number of records
  - Each name should be unique

- List of unique diseases can be fetched from the **icd10 dataset**


In [ ]:
census_data = pd.read_excel('/content/Names_2010Census_Top1000.xlsx')

In [ ]:
census_data.columns = census_data.loc[1]
census_data.drop([0, 1], inplace = True)
census_data.reset_index(drop = True, inplace=True)
census_data.head()

1,SURNAME,RANK,FREQUENCY (COUNT),"PROPORTION PER 100,000 POPULATION",CUMULATIVE PROPORTION,PERCENT NON-HISPANIC OR LATINO WHITE ALONE,PERCENT NON-HISPANIC OR LATINO BLACK OR AFRICAN AMERICAN ALONE,PERCENT NON-HISPANIC OR LATINO ASIAN AND NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE,PERCENT NON-HISPANIC OR LATINO AMERICAN INDIAN AND ALASKA NATIVE ALONE,PERCENT NON-HISPANIC OR LATINO TWO OR MORE RACES,PERCENT HISPANIC OR LATINO ORIGIN
0,SMITH,1,2442977,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4
1,JOHNSON,2,1932812,655.24,1483.42,58.97,34.63,0.54,0.94,2.56,2.36
2,WILLIAMS,3,1625252,550.97,2034.39,45.75,47.68,0.46,0.82,2.81,2.49
3,BROWN,4,1437026,487.16,2521.56,57.95,35.6,0.51,0.87,2.55,2.52
4,JONES,5,1425470,483.24,3004.8,55.19,38.48,0.44,1,2.61,2.29


In [ ]:
surname = census_data.SURNAME.tolist()

# sur = census_data['SURNAME'].tolist()
print(surname)

['SMITH', 'JOHNSON', 'WILLIAMS', 'BROWN', 'JONES', 'GARCIA', 'MILLER', 'DAVIS', 'RODRIGUEZ', 'MARTINEZ', 'HERNANDEZ', 'LOPEZ', 'GONZALEZ', 'WILSON', 'ANDERSON', 'THOMAS', 'TAYLOR', 'MOORE', 'JACKSON', 'MARTIN', 'LEE', 'PEREZ', 'THOMPSON', 'WHITE', 'HARRIS', 'SANCHEZ', 'CLARK', 'RAMIREZ', 'LEWIS', 'ROBINSON', 'WALKER', 'YOUNG', 'ALLEN', 'KING', 'WRIGHT', 'SCOTT', 'TORRES', 'NGUYEN', 'HILL', 'FLORES', 'GREEN', 'ADAMS', 'NELSON', 'BAKER', 'HALL', 'RIVERA', 'CAMPBELL', 'MITCHELL', 'CARTER', 'ROBERTS', 'GOMEZ', 'PHILLIPS', 'EVANS', 'TURNER', 'DIAZ', 'PARKER', 'CRUZ', 'EDWARDS', 'COLLINS', 'REYES', 'STEWART', 'MORRIS', 'MORALES', 'MURPHY', 'COOK', 'ROGERS', 'GUTIERREZ', 'ORTIZ', 'MORGAN', 'COOPER', 'PETERSON', 'BAILEY', 'REED', 'KELLY', 'HOWARD', 'RAMOS', 'KIM', 'COX', 'WARD', 'RICHARDSON', 'WATSON', 'BROOKS', 'CHAVEZ', 'WOOD', 'JAMES', 'BENNETT', 'GRAY', 'MENDOZA', 'RUIZ', 'HUGHES', 'PRICE', 'ALVAREZ', 'CASTILLO', 'SANDERS', 'PATEL', 'MYERS', 'LONG', 'ROSS', 'FOSTER', 'JIMENEZ', 'POWELL', '

In [ ]:
def generate_canaries(names, diseases, num=200):
  canaries = []

  return canaries

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 211269
    })
})

In [ ]:
def insert_canaries(data: pd.DataFrame, canaries: list):


In [ ]:
data = pd.DataFrame(dataset['train'], columns = dataset['train'].features)

In [ ]:
data.loc[0, 'long_answer']

'As ILC2s are elevated in patients with CRSwNP, they may drive nasal polyp formation in CRS. ILC2s are also linked with high tissue and blood eosinophilia and have a potential role in the activation and survival of eosinophils during the Th2 immune response. The association of innate lymphoid cells in CRS provides insights into its pathogenesis.'